In [ ]:
# Operator Learning via Kernel Methods
#
# This is mostly a script for prototyping methods

In [ ]:
# Imports
import numpy as np
import sklearn as skl
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.kernel_ridge import KernelRidge
import sys
sys.path.append('./')
from KLearn import *
from functools import reduce
import matplotlib.pyplot as plt

In [ ]:
# Define some Kernels
gamma = 50;
K = lambda y1,y2 : rbf_kernel(y1,y2, gamma = gamma)
S = lambda x1,x2 : rbf_kernel(x1.T,x2.T, gamma = 0.00001) # lame, for now

In [ ]:
### Generating Data

## GP kernel

def gp_kernel(t, sigma):
  return np.exp( - (1/(2*sigma**2))*t**2)

N_mesh = 100
x = np.linspace(0,1, N_mesh) # fine grid for visualization

## compute pairwise distance matrix

dist_matrix = pairwise_distances( x.reshape(-1,1), x.reshape(-1,1))

## GP lengthscale
sigma = 0.1
Cov_matrix = gp_kernel(dist_matrix, sigma) + 1e-8*np.eye(N_mesh)
Cov_matrix = 1/2*(Cov_matrix + Cov_matrix.T)

Cov_matrix_sqrt = np.linalg.cholesky(Cov_matrix)

print(Cov_matrix_sqrt)

In [ ]:
## Training and test functions

N_train = 1000    # Number of training functions
N_test  = 5     # Number of testing functions
u_train = np.zeros((N_mesh, N_train))
u_test  = np.zeros((N_mesh, N_test))

for i in range(N_train):
  xi = np.random.randn(N_mesh)
  u_train[:, i] = np.dot(Cov_matrix_sqrt, xi)

for i in range(N_test):
  xi = np.random.randn(N_mesh)
  u_test [:, i] = np.dot(Cov_matrix_sqrt, xi)
    
u_train = u_train*1./np.linalg.norm(u_train, axis=0)
u_test = u_test*1./np.linalg.norm(u_test, axis=0)
    
# define pointwise operator

def op_square(u):
  return u**2

def op_sin(u):
    return np.sin(u)

def op_I(u):
    return u

def op_diff(u,x):
    return np.gradient(u, x, axis=0)

v_train = op_square(u_train)
v_test  = op_square(u_test)

In [ ]:
# Finish setting up
U = u_train;
V = v_train;
Y = x.reshape((-1, 1));

print(np.shape(U))
print(U)
print(np.shape(V))
print(np.shape(Y))

In [ ]:
# LEARN!
D,f = KLearn(U,V,N_mesh,S,K,Y)

In [ ]:
# Does it work... ?
v_mod = D(u_test);
np.max(np.abs(v_mod(Y) - v_test)) # very nice...

In [ ]:
# Define some Kernels again (this time, parameterized)
K_ = lambda g: (lambda y1,y2 : rbf_kernel(y1,y2, gamma = g))
S_ = lambda s : (lambda x1,x2 : rbf_kernel(x1,x2, gamma = s)) # lame, for now
kgrid = np.linspace(10,200, 100)
sgrid = 1/np.power(1.8,range(1,50))

In [ ]:
# LEARN AGAIN!
D2,scoresS,gS = OpLearn(U,V, S_,sgrid, K_, kgrid,Y, report_scores=True)
print(scores) # THESE ARE THE CV SCORES (currently rel. MSE)
v_mod2,scoresK,gK = D2(u_test);

In [ ]:
# Let's plot the errors
fig, ax = plt.subplots(1,2)
fig.set_size_inches(10,5)
ax[0].loglog(sgrid, scoresS)
ax[0].set( xlabel ='$\gamma_s$', ylabel='CV Score', title = "Parameter tuning for Recovery Map Kernel S")

ax[1].loglog(kgrid, scoresK)
ax[1].set( xlabel ='$\gamma_k$', ylabel='CV Score', title = "Parameter tuning for Operator Kernel K")

In [ ]:
# Does it work... ?
np.max(np.abs(v_mod2(Y) - v_test)) # very nice...

In [ ]:
fig, ax = plt.subplots(2,2)
fig.set_size_inches(25,15)

ax[0,0].plot(u_test)
ax[0,0].set( xlabel ='$x$', ylabel='$u_i(x)$', title = "Input Test Functions")

ax[1,0].plot(v_test)
ax[1,0].set( xlabel ='$x$', ylabel='$v_i(x) = \mathcal{G}^\dagger (u_i(x))$', title = "Output Test Functions")

ax[1,1].plot(v_mod2(Y))
ax[1,1].set( xlabel ='$x$', ylabel='$v^*_i(x) = \mathcal{G}^*(u_i(x))$', title = "Learned Test Functions")

ax[0,1].plot(v_mod2(Y)-v_test)
ax[0,1].set( xlabel ='$x$', ylabel='$|\mathcal{G}^*(u_i(x)) - v^\dagger_i(x)|$' , title = "CV-trained Absolute Error")

fig.savefig('Diff_Operator.png', dpi=300, bbox_inches='tight')